In [ ]:
import cv2
import numpy as np
from collections import deque

In [ ]:
def img_resize(img,width=None,height=None,inter=cv2.INTER_AREA):
    dim=None
    (h,w)=img.shape[:2]
    if width is None and height is None:
        return
    if width is None:
        r=height/float(h)
        dim=(int(w*r),height)
    else:
        r=width/float(w)
        dim=(width,int(h*r))
    return cv2.resize(img,dim,interpolation=inter)

In [ ]:
pts=deque(maxlen=64)
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    frame=img_resize(frame,width=650)
    frame=cv2.GaussianBlur(frame,(11,11),0)
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    HSVLOW = np.array([35,56,51])
    HSVHIGH = np.array([92,204,140])
    mask=cv2.inRange(hsv, HSVLOW, HSVHIGH)
    
    kernal=np.ones((3,3))
    mask=cv2.dilate(mask,kernal,iterations=2)
    mask=cv2.erode(mask,kernal,iterations=2)
    
    cnts,hier=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    center=None
    if len(cnts)>0:
        c=max(cnts,key=cv2.contourArea)
        ((x,y),r)=cv2.minEnclosingCircle(c)
        M=cv2.moments(c)
        center=(int(M['m10']/M['m00']),int(M['m01']/M['m00']))
        if r>10:
            cv2.circle(frame,(int(x),int(y)),int(r),(0,0,0),2)
            cv2.circle(frame,center,5,(0,0,255),-1)
        pts.appendleft(center)
    for i in range(1,len(pts)):
        if pts[i-1] is None or pts[i] is None:
            continue
        thickness=int((np.sqrt(64/float(i+1)))*2.5)
        cv2.line(frame,pts[i-1],pts[i],(0,0,255),thickness)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1)&0xFF==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()



In [ ]:
cv2.destroyAllWindows()